In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv

import os
print(os.listdir("../input"))

%load_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.torch_imports import *
from fastai.io import *
import torch.nn as nn


['train.csv', 'sample_submission.csv', 'test.csv']


/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
trn, test = pd.read_csv('../input/train.csv'), pd.read_csv('../input/test.csv')
trn.shape, test.shape

((42000, 785), (28000, 784))

In [3]:
x0, y0 = trn.drop(['label'], axis =1).values, trn['label'].values
x0.shape, y0.shape

((42000, 784), (42000,))

In [4]:
mean= x0.mean()
std = x0.std()

In [5]:
x0= (x0-mean)/std

In [15]:
from sklearn.model_selection import train_test_split
x,x_valid, y, y_valid = train_test_split(x0, y0, test_size=0.02,random_state=42)
x.shape, x_valid.shape, y.shape

((41160, 784), (840, 784), (41160,))

In [16]:
x_trn, y_trn = torch.from_numpy(x), torch.from_numpy(y)
type(x_trn)

torch.Tensor

In [17]:
net = nn.Sequential(
    nn.Linear(28*28, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 10),
    nn.LogSoftmax()
).cuda()

In [18]:
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *

md = ImageClassifierData.from_arrays('tmp/tmp/', (x,y), (x_valid, y_valid))

In [19]:
loss=nn.NLLLoss()
metrics=[accuracy]
# opt=optim.SGD(net.parameters(), 1e-1, momentum=0.9)
opt=optim.SGD(net.parameters(), 1e-1, momentum=0.9, weight_decay=1e-3)

In [20]:
fit(net, md, n_epochs=5, crit=loss, opt=opt, metrics=metrics)

epoch      trn_loss   val_loss   accuracy                     
    0      0.265657   0.261075   0.942857  
    1      0.23299    0.207016   0.942857                     
    2      0.196236   0.192246   0.945238                     
    3      0.208708   0.222976   0.944048                     
    4      0.18785    0.250895   0.929762                     



[0.2508953355607532, 0.9297619047619048]

In [21]:
set_lrs(opt, 1e-2)

fit(net, md, n_epochs=3, crit=loss, opt=opt, metrics=metrics)

epoch      trn_loss   val_loss   accuracy                      
    0      0.071243   0.103296   0.966667  
    1      0.048618   0.096822   0.969048                      
    2      0.042031   0.087243   0.97381                       



[0.08724256384940375, 0.9738095238095238]

In [ ]:
fit(net, md, n_epochs=6, crit=loss, opt=opt, metrics=metrics)

  0%|          | 0/644 [00:00<?, ?it/s, loss=0.0165]

In [23]:
test_data=(test-mean)/std